In [1]:
import IPython.display as ipd
import numpy as np
from datasets import load_dataset

from dataset import concatenate

## Dataset Exploration: 

In [2]:
ds = load_dataset("edinburghcstr/ami", "ihm", split="train")

dataset = ds.filter(lambda x: x["meeting_id"] == "EN2001a")
dataset = dataset.sort("begin_time")
dataset = dataset.select(range(320))
result = dataset.map(
    concatenate, batched=True, batch_size=32, remove_columns=dataset.column_names
)

/home/kamil/projects/pyannote-audio/env/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for edinburghcstr/ami contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/edinburghcstr/ami
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [28]:
result

Dataset({
    features: ['audio', 'speakers', 'timestamps_start', 'timestamps_end'],
    num_rows: 10
})

## Raw AMI dataset: 

In [20]:
import os 

path_to_ami = "/home/kamil/projects/AMI-diarization-setup/"
path_to_ami_audio_files = os.path.join(path_to_ami, "pyannote/amicorpus")
path_to_ami_rttms = os.path.join(path_to_ami, "only_words/rttms/train")

In [21]:
os.path.join(path_to_ami_rttms, '')

'/home/kamil/projects/AMI-diarization-setup/only_words/rttms/train/'

## Processed AMI dataset: 

In [18]:
from pyannote.database import registry, FileFinder
from pyannote.audio.tasks import SpeakerDiarization, VoiceActivityDetection
from pyannote.audio.models.segmentation import PyanNet
import pytorch_lightning as pl

registry.load_database("/home/kamil/projects/AMI-diarization-setup/pyannote/database.yml")
ami = registry.get_protocol('AMI.SpeakerDiarization.mini')

/home/kamil/projects/pyannote-audio/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


'AMI-SDM.SpeakerDiarization.only_words' found in /home/kamil/projects/AMI-diarization-setup/pyannote/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
'AMI-SDM.SpeakerDiarization.mini' found in /home/kamil/projects/AMI-diarization-setup/pyannote/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
'AMI.SpeakerDiarization.only_words' found in /home/kamil/projects/AMI-diarization-setup/pyannote/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
'AMI.SpeakerDiarization.mini' found in /home/kamil/projects/AMI-diarization-setup/pyannote/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
'AMI.SpeakerDiarization.word_and_vocalsounds' found in /home/kamil/projects/AMI-diarization-setup/pyannote/database.yml does not define the 'scope' of speaker labels (file, dat

In [23]:
spk_task = SpeakerDiarization(ami, duration=2.0, batch_size=128)

Protocol AMI.SpeakerDiarization.mini does not provide the path to audio files: adding an 'audio' preprocessor for you. See pyannote.database documentation on how to do that yourself.
Protocol AMI.SpeakerDiarization.mini does not precompute the output of torchaudio.info(): adding a 'torchaudio.info' preprocessor for you to speed up dataloaders. See pyannote.database documentation on how to do that yourself.


In [24]:
import itertools 

files_iter = spk_task.protocol.train()

for file_id, file in enumerate(files_iter): 
    print(file_id, file)
    break

0 <pyannote.database.protocol.protocol.ProtocolFile object at 0x7f3475eb3a60>
